In [117]:
from astropy.coordinates import EarthLocation,SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
from astropy.coordinates import AltAz
from astropy.time import Time, TimeDelta
from astropy import units as u
import numpy as np
dt = TimeDelta(1, format='jd')
t=Time('2023-10-01T11:00:00',format="isot",scale="utc")+ dt * np.linspace(0., 1., 13)
breakout=lambda la,lo,ht:{'lat':la,'lon':lo,'height':ht*u.m}

def xyz2uvwMAT(Ra,Dec):
    sinH,cosH=(np.sin(Ra),np.cos(Ra))
    sinI,cosI=(np.sin(Dec),np.cos(Dec))
    return np.array([[ sinH     ,cosH     ,0],
                     [-sinI*cosH,sinI*sinH,cosI],
                     [cosI*cosH,-cosI*sinH,sinI]])

earthSpots={"Bunker Hill Monument":[42.37630000,-71.06070000,30.0,1531609,-4463479,4276597,844.3,932.3,20.0],
            "Old South Church":[42.35020000,-71.07800000,20.0,1530895,-4465783,4274447,-579.0,-1968.4,10.0],
            "Old North Church":[42.36640000,-71.05440000,20.0,1532341,-4464003,4275779,1366.7,-166.4,10.0],
            "USS Constitution":[42.37250000,-71.05660000,5.0,1532016,-4463622,4276267,1182.5,508.9,-5.0],
            "Museum of Science":[42.36790000,-71.07100000,10.0,1531010,-4464333,4275895,0.0,0.0,0.0],
            "Kendall Square":[42.36570000,-71.08230000,7.0,1530181,-4464795,4275706,-934.4,-252.6,-3.0],
            "Hatch Shell":[42.35790000,-71.07390000,4.0,1531021,-4465118,4275068,-244.4,-1113.7,-6.0],
            "Frog Pond":[42.35620000,-71.06580000,15.0,1531700,-4465028,4274936,427.7,-1302.1,5.0],
            "Public Garden":[42.35400000,-71.07000000,8.0,1531421,-4465292,4274751,77.8,-1545.9,-2.0],
            "Public Library":[42.34950000,-71.07820000,20.0,1530898,-4465841,4274385,-594.8,-2052.0,10.0],
            "MIT Dome":[42.35990000,-71.09200000,40.0,1529573,-4465489,4275251,-1733.5,-898.5,30.0]}
             
print (f"Location Baselines:{len(earthSpots)*(len(earthSpots)-1)/2}")

observingFrequency=1.42e9     #Hz
c=299792458                   #m/s
observingWavelength=c/observingFrequency
print(f"Observing Freq and wavelength: {observingFrequency/1e6} MHz   {observingWavelength*100:5.4} cm")
#phase reference position (RA, DEC Degrees)
Ho=82.9106369047619
Io=1.077503968253968
HoRAD,IoRAD = np.array([Ho,Io])*np.pi/180.0
print(f"Phase Ref Position: Ho={Ho:5.4f}    Io={Io:5.4f} (degrees)")
print(f"                    Ho={HoRAD:5.4f}     Io={IoRAD:5.4f} (radians)")
baselines=[((a,EarthLocation(**breakout(*(earthSpots[a][:3])))),(b,EarthLocation(**breakout(*(earthSpots[b][:3]))))) for i,a in enumerate(earthSpots.keys()) for j,b in enumerate(earthSpots.keys()) if j>i]
print(f"Baselines: ({len(baselines)} total) {baselines}")
#XYZp,XYZv=observing_location.get_gcrs_posvel(t[0] + dt * np.linspace(0., 1., 13))
#XYZp



Location Baselines:55.0
Observing Freq and wavelength: 1420.0 MHz   21.11 cm
Phase Ref Position: Ho=82.9106    Io=1.0775 (degrees)
                    Ho=1.4471     Io=0.0188 (radians)
Baselines: (55 total) [(('Bunker Hill Monument', <EarthLocation (1531612.29777372, -4463480.79182169, 4276594.14281663) m>), ('Old South Church', <EarthLocation (1530895.62456799, -4465784.05712956, 4274445.21520773) m>)), (('Bunker Hill Monument', <EarthLocation (1531612.29777372, -4463480.79182169, 4276594.14281663) m>), ('Old North Church', <EarthLocation (1532341.29648784, -4464006.34382596, 4275774.95252306) m>)), (('Bunker Hill Monument', <EarthLocation (1531612.29777372, -4463480.79182169, 4276594.14281663) m>), ('USS Constitution', <EarthLocation (1532018.05287924, -4463622.79255069, 4276265.4589869) m>)), (('Bunker Hill Monument', <EarthLocation (1531612.29777372, -4463480.79182169, 4276594.14281663) m>), ('Museum of Science', <EarthLocation (1531009.07873295, -4464336.9143563, 4275891.32031223)

In [150]:
EarthLocation.from_geocentric(**dict((ff,(x1.value+x2.value)/2) for x1,x2,ff in zip(baselines[0][0][1].to_geocentric() , baselines[0][1][1].to_geocentric(),['x','y','z'])),unit=u.m)

<EarthLocation (1531253.96117086, -4464632.42447563, 4275519.67901218) m>

In [94]:
# Bunker Hill Monument: 42.37633621914482, -71.06072544082996
observing_location = EarthLocation(**breakout(*(earthSpots['Bunker Hill Monument'][:3])))  
#observing_location = EarthLocation(lat='42.37633621914482', lon='-71.06072544082996', height=30*u.m)  
observing_time = t[0]  
aa = AltAz(location=observing_location, obstime=observing_time)

#Rigel and Betelgeuse
coords = ["05:14:33.700000	-8.200806", "05:55:11.700000	7.407667"]
c = SkyCoord(coords, unit=(u.hourangle, u.deg))
#print(c)
#coord = SkyCoord('4h42m', '-38d6m50.8s')
print(f"SkyCoord original: {c}")
print(f"Transformed to AA:{c.transform_to(aa)}")


SkyCoord original: <SkyCoord (ICRS): (ra, dec) in deg
    [(78.64041667, -8.200806), (88.79875   ,  7.407667)]>
Transformed to AA:<SkyCoord (AltAz: obstime=2023-10-01T11:00:00.000, location=(1531612.29777372, -4463480.79182169, 4276594.14281663) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    [(210.44824942, 34.56128858), (204.60635023, 52.69407351)]>


In [82]:
XYZp,XYZv=observing_location.get_gcrs_posvel(Time.now())
print(f"   Position: {','.join([f'{int(xx.value)}' for xx in XYZp.xyz])} m\n   Velocity: {XYZv}")

   Position: 4474269,-1528900,4266278 m
   Velocity: (111.49887857, 325.54613009, -0.26916088) m / s


In [80]:
[f'{int(xx.value)}' for xx in XYZp.xyz]

['4462406', '-1563109', '4266306']

In [92]:
def xyz2uvwMAT(Ra,Dec):
    sinH,cosH=(np.sin(Ra),np.cos(Ra))
    sinI,cosI=(np.sin(Dec),np.cos(Dec))
    return np.array([[ sinH     ,cosH     ,0],
                     [-sinI*cosH,sinI*sinH,cosI],
                     [cosI*cosH,-cosI*sinH,sinI]])

xyz2uvwMAT(np.pi/3,np.pi/4)


array([[ 0.8660254 ,  0.5       ,  0.        ],
       [-0.35355339,  0.61237244,  0.70710678],
       [ 0.35355339, -0.61237244,  0.70710678]])

In [112]:
XYZp,XYZv=observing_location.get_gcrs_posvel(Time('2023-10-01T17:00:00',format="isot",scale="utc") + dt * np.linspace(0., 1., 13))
XYZp

<CartesianRepresentation (x, y, z) in m
    [(-4572789.36164484, -1125968.34879062, 4287137.41365681),
     (-3391091.02296403, -3271276.8499555 , 4284507.42990888),
     (-1293244.73663035, -4535311.92940957, 4279745.11228876),
     ( 1155622.27611058, -4577562.17527132, 4274133.38099184),
     ( 3295823.26004819, -3386646.04968133, 4269184.00221044),
     ( 4550821.29017248, -1283377.90053092, 4266230.32915104),
     ( 4582539.38078458,  1165654.43420227, 4266068.10020767),
     ( 3382433.18942446,  3300719.60883963, 4268741.06206125),
     ( 1273792.72618119,  4546664.15871351, 4273529.17278264),
     (-1175346.95907428,  4567849.92114224, 4279142.5654846 ),
     (-3305225.60648325,  3358569.75005321, 4284069.02328865),
     (-4542086.92486029,  1244584.9332676 , 4286981.36022573),
     (-4552739.63574187, -1204629.84408784, 4287094.96278124)]>

In [108]:
import numpy as np
t=[Time('2023-10-01T17:00:00',format="isot",scale="utc")]
dt = TimeDelta(1, format='jd')
t[0] + dt * np.linspace(0., 1., 13)

<Time object: scale='utc' format='isot' value=['2023-10-01T17:00:00.000' '2023-10-01T19:00:00.000'
 '2023-10-01T21:00:00.000' '2023-10-01T23:00:00.000'
 '2023-10-02T01:00:00.000' '2023-10-02T03:00:00.000'
 '2023-10-02T05:00:00.000' '2023-10-02T07:00:00.000'
 '2023-10-02T09:00:00.000' '2023-10-02T11:00:00.000'
 '2023-10-02T13:00:00.000' '2023-10-02T15:00:00.000'
 '2023-10-02T17:00:00.000']>

AssertionError: 